# Week 11: Reading Data for Different Recording System

Source: [MNE-Python](https://mne.tools/stable/index.html)

## 1. Importing Data from MEG Device

This section describes how to read data for various MEG manufacturers.

### MEGIN/Elekta Neuromag VectorView and TRIUX (.fif)

Neuromag Raw FIF files can be loaded using `mne.io.read_raw_fif()`.

If the data were recorded with MaxShield on and have not been processed with MaxFilter, they may need to be loaded with `mne.io.read_raw_fif(..., allow_maxshield=True)`.

### FIL OPM (.bin)

MEG data from the OPM system used by the FIL at UCL can be read with `mne.io.read_raw_fil()`. For related OPM processing methods, see Preprocessing optically pumped magnetometer (OPM) MEG data.

### Artemis123 (.bin)

MEG data from the Artemis123 system can be read with `mne.io.read_raw_artemis123()`.

### 4-D Neuroimaging / BTI data (dir)

MNE-Python provides `mne.io.read_raw_bti()` to read and convert 4D / BTI data. This reader function will by default replace the original channel names, typically composed of the letter `A` and the channel number with Neuromag. To import the data, the following input files are mandatory:

* A data file (typically c,rfDC) containing the recorded MEG time series.

* A hs_file containing the digitizer data.

* A config file containing acquisition information and metadata.

By default `mne.io.read_raw_bti()` assumes that these three files are located in the same folder.

<div class="alert alert-info"><h4>Note</h4><p>While reading the reference or compensation channels,
          the compensation weights are currently not processed.
          As a result, the :class:`mne.io.Raw` object and the corresponding fif
          file does not include information about the compensation channels
          and the weights to be applied to realize software gradient
          compensation. If the data are saved in the Magnes system are already
          compensated, there will be a small error in the forward calculations,
          whose significance has not been evaluated carefully at this time.</p></div>

### CTF data (dir)

The function `mne.io.read_raw_ctf()` can be used to read CTF data.

### CTF Polhemus data

The function `mne.channels.read_dig_polhemus_isotrak()` can be used to read Polhemus data.

### Applying software gradient compensation

Since the software gradient compensation employed in CTF systems is a reversible operation, it is possible to change the compensation status of CTF data in the data files as desired. This section contains information about the technical details of the compensation procedure and a description of `mne.io.Raw.apply_gradient_compensation()`.

The raw instances returned by `mne.io.read_raw_ctf()` contain several compensation matrices which are employed to suppress external disturbances with help of the reference channel data. The reference sensors are located further away from the brain than the helmet sensors and are thus measuring mainly the external disturbances rather than magnetic fields originating in the brain. Most often, a compensation matrix corresponding to a scheme nicknamed *Third-order gradient compensation* is employed.

Let us assume that the data contain $n_1$ MEG
sensor channels, $n_2$ reference sensor
channels, and $n_3$ other channels.
The data from all channels can be concatenated into a single vector

\begin{align}x = [x_1^T x_2^T x_3^T]^T\ ,\end{align}

where $x_1$, $x_2$,
and $x_3$ are the data vectors corresponding
to the MEG sensor channels, reference sensor channels, and other
channels, respectively. The data before and after compensation,
denoted here by $x_{(0)}$ and $x_{(k)}$, respectively,
are related by

\begin{align}x_{(k)} = M_{(k)} x_{(0)}\ ,\end{align}

where the composite compensation matrix is

\begin{align}M_{(k)} = \begin{bmatrix}
                I_{n_1} & C_{(k)} & 0 \\
                0 & I_{n_2} & 0 \\
                0 & 0 & I_{n_3}
                \end{bmatrix}\ .\end{align}

In the above, $C_{(k)}$ is a $n_1$ by $n_2$ compensation
data matrix corresponding to compensation "grade" $k$.
It is easy to see that

\begin{align}M_{(k)}^{-1} = \begin{bmatrix}
                I_{n_1} & -C_{(k)} & 0 \\
                0 & I_{n_2} & 0 \\
                0 & 0 & I_{n_3}
                \end{bmatrix}\ .\end{align}

To convert from compensation grade $k$ to $p$ one
can simply multiply the inverse of one compensate compensation matrix
by another and apply the product to the data:

\begin{align}x_{(k)} = M_{(k)} M_{(p)}^{-1} x_{(p)}\ .\end{align}

This operation is performed by :meth:`mne.io.Raw.apply_gradient_compensation`.

### Ricoh/KIT MEG system data (.con/.sqd)

MNE-Python includes the `mne.io.read_raw_kit()` and `mne.read_epochs_kit()` to read and convert Ricoh/KIT MEG data.

**Note (Channel Naming)**:
```
In MNE 0.21 This reader function will by default replace the original channel names, which typically with index starting with zero, with ones with an index starting with one. In 0.22 it will use native names when possible. Use the standardize_names argument to control this behavior.
```

To import continuous data, only the input .sqd or .con file is needed. For epochs, an Nx3 matrix containing the event number/corresponding trigger value in the third column is needed.

The following input files are optional:

* A KIT marker file (mrk file) or an array-like containing the locations of the HPI coils in the MEG device coordinate system. These data are used together with the elp file to establish the coordinate transformation between the head and device coordinate systems.

* A Polhemus points file (elp file) or an array-like containing the locations of the fiducials and the head-position indicator (HPI) coils. These data are usually given in the Polhemus head coordinate system.

* A Polhemus head shape data file (hsp file) or an array-like containing locations of additional points from the head surface. These points must be given in the same coordinate system as that used for the elp file.

Modern Ricoh systems may encode this information it the file itself, in which case `mrk`, `elp`, and `hsp` can all be `None` and the data will be read from the file itself.

<div class="alert alert-info"><h4>Note</h4><p>The output fif file will use the Neuromag head coordinate system convention,
   see `coordinate_systems`. A coordinate transformation between the
   Polhemus head coordinates and the Neuromag head coordinates is included.</p></div>

By default, KIT-157 systems assume the first 157 channels are the MEG channels,
the next 3 channels are the reference compensation channels, and channels 160
onwards are designated as miscellaneous input channels (MISC 001, MISC 002,
etc.).
By default, KIT-208 systems assume the first 208 channels are the MEG channels,
the next 16 channels are the reference compensation channels, and channels 224
onwards are designated as miscellaneous input channels (MISC 001, MISC 002,
etc.).

In addition, it is possible to synthesize the digital trigger channel (STI 014)
from available analog trigger channel data by specifying the following
parameters:

- A list of trigger channels (stim) or default triggers with order: '<' | '>'
  Channel-value correspondence when converting KIT trigger channels to a
  Neuromag-style stim channel. By default, we assume the first eight
  miscellaneous channels are trigger channels. For '<', the largest values are
  assigned to the first channel (little endian; default). For '>', the largest
  values are assigned to the last channel (big endian). Can also be specified
  as a list of trigger channel indexes.
- The trigger channel slope (slope) : '+' | '-'
  How to interpret values on KIT trigger channels when synthesizing a
  Neuromag-style stim channel. With '+', a positive slope (low-to-high)
  is interpreted as an event. With '-', a negative slope (high-to-low)
  is interpreted as an event.
- A stimulus threshold (stimthresh) : float
  The threshold level for accepting voltage changes in KIT trigger
  channels as a trigger event.

The synthesized trigger channel data value at sample $k$ will
be:

\begin{align}s(k) = \sum_{p = 1}^n {t_p(k) 2^{p - 1}}\ ,\end{align}

where $t_p(k)$ are the thresholded
from the input channel data d_p(k):

\begin{align}t_p(k) = \Bigg\{ \begin{array}{l}
                 0 \text{  if  } d_p(k) \leq t\\
                 1 \text{  if  } d_p(k) > t
             \end{array}\ .\end{align}

The threshold value $t$ can
be adjusted with the ``stimthresh`` parameter.

### FieldTrip MEG/EEG data (.mat)

MNE-Python includes :func:`mne.io.read_raw_fieldtrip`, :func:`mne.read_epochs_fieldtrip` and :func:`mne.read_evoked_fieldtrip` to read data coming from FieldTrip.

The data is imported directly from a ``.mat`` file.

The ``info`` parameter can be explicitly set to ``None``. The import functions will still work but:

1. All channel locations will be in head coordinates.
2. Channel orientations cannot be guaranteed to be accurate.
3. All channel types will be set to generic types.

This is probably fine for anything that does not need that information, but if you intent to do things like interpolation of missing channels, source analysis or look at the RMS pairs of planar gradiometers, you most likely run into problems.

It is **highly recommended** to provide the `info` parameter as well. The `info` dictionary can be extracted by loading the original raw data file with the corresponding MNE-Python functions:

In [ ]:
original_data = mne.io.read_raw_fiff('original_data.fif', preload=False)
original_info = original_data.info
data_from_ft = mne.read_evoked_fieldtrip('evoked_data.mat', original_info)

The imported data can have less channels than the original data. Only the information for the present ones is extracted from the `info` dictionary.

As of version 0.17, importing FieldTrip data has been tested on a variety of systems with the following results:


**Correct the Table** ###################
| System | Read Raw Data | Read Epoched Data | Read Evoked Data |
| ------ | ------------- | ----------------- | ---------------- |
| BTI | Works | Untested | Untested |
| CNT |Data imported as microvolts.  Otherwise fine.| Data imported as microvolts. Otherwise fine. | Data imported as microvolts. Otherwise fine. |

**Correct the Table** ####################

### Creating MNE data structures from arbitrary data (from memory)

Arbitrary (e.g., simulated or manually read in) raw data can be constructed from memory by making use of `mne.io.RawArray`, `mne.EpochsArray` or `mne.EvokedArray` in combination with `mne.create_info()`.

This functionality is illustrated in `Creating MNE-Python data structures from scratch`. Using 3rd party libraries such as `NEO` in combination with these functions abundant electrophysiological file formats can be easily loaded into MNE.

## 2. Importing Data from EEG Device

MNE includes various functions and utilities for reading EEG data and electrode locations.

### BrainVision (.vhdr, .vmrk, .eeg)

The BrainVision file format consists of three separate files:

1. A text header file (``.vhdr``) containing meta data.
2. A text marker file (``.vmrk``) containing information about events in the
   data.
3. A binary data file (``.eeg``) containing the voltage values of the EEG.

Both text files are based on the [INI format](https://en.wikipedia.org/wiki/INI_file)
consisting of

* sections marked as ``[square brackets]``,
* comments marked as ``; comment``,
* and key-value pairs marked as ``key=value``.

Brain Products provides documentation for their core BrainVision file format.
The format specification is hosted on the
[Brain Products website](https://www.brainproducts.com/support-resources/brainvision-core-data-format-1-0/).

BrainVision EEG files can be read using `mne.io.read_raw_brainvision`,
passing the ``.vhdr`` header file as the argument.

<div class="alert alert-danger"><h4>Warning</h4><p>Renaming BrainVision files can be problematic due to their
             multi-file structure. See this
             [example](https://mne.tools/mne-bids/stable/auto_examples/rename_brainvision_files.html#sphx-glr-auto-examples-rename-brainvision-files-py)
             for instructions.</p></div>

<div class="alert alert-info"><h4>Note</h4><p>For *writing* BrainVision files, take a look at the :mod:`mne.export`
          module, which used the [pybv](https://pypi.org/project/pybv/) Python
          package.</p></div>

### European data format (.edf)

[EDF](http://www.edfplus.info/specs/edf.html) and
[EDF+](http://www.edfplus.info/specs/edfplus.html) files can be read using
`mne.io.read_raw_edf`. Both variants are 16-bit formats.

EDF+ files may contain annotation channels which can be used to store trigger
and event information. These annotations are available in ``raw.annotations``.

Writing EDF files is not supported natively yet. [This gist](https://gist.github.com/skjerns/bc660ef59dca0dbd53f00ed38c42f6be)_ or
[MNELAB](https://github.com/cbrnr/mnelab) (both of which use
[pyedflib](https://github.com/holgern/pyedflib) under the hood) can be used
to export any `mne.io.Raw` object to EDF/EDF+/BDF/BDF+.

### BioSemi data format (.bdf)

The `BDF format` is a 24-bit variant of the EDF format used by EEG systems manufactured by BioSemi. It can be imported with `mne.io.read_raw_bdf()`.

BioSemi amplifiers do not perform “common mode noise rejection” automatically. The signals in the EEG file are the voltages between each electrode and the CMS active electrode, which still contain some CM noise (50 Hz, ADC reference noise, etc.). The [BioSemi FAQ](https://www.biosemi.com/faq/cms&drl.htm) provides more details on this topic. Therefore, it is advisable to choose a reference (e.g., a single channel like Cz, average of linked mastoids, average of all electrodes, etc.) after importing BioSemi data to avoid losing signal information. The data can be re-referenced later after cleaning if desired.

<div class="alert alert-danger"><h4>Warning</h4><p>Data samples in a BDF file are represented in a 3-byte
             (24-bit) format. Since 3-byte raw data buffers are not presently
             supported in the FIF format, these data will be changed to 4-byte
             integers in the conversion.</p></div>
             
### General data format (.gdf)

GDF files can be read using `mne.io.read_raw_gdf`.

[GDF (General Data Format)](https://arxiv.org/abs/cs/0608052) is a flexible
format for biomedical signals that overcomes some of the limitations of the
EDF format. The original specification (GDF v1) includes a binary header
and uses an event table. An updated specification (GDF v2) was released in
2011 and adds fields for additional subject-specific information (gender,
age, etc.) and allows storing several physical units and other properties.
Both specifications are supported by MNE.

### Neuroscan CNT (.cnt)

CNT files can be read using `mne.io.read_raw_cnt`.
Channel locations can be read from a montage or the file header. If read
from the header, the data channels (channels that are not assigned to EOG, ECG,
EMG or MISC) are fit to a sphere and assigned a z-value accordingly. If a
non-data channel does not fit to the sphere, it is assigned a z-value of 0.

<div class="alert alert-danger"><h4>Warning</h4><p>Reading channel locations from the file header may be dangerous, as the
    x_coord and y_coord in the ELECTLOC section of the header do not necessarily
    translate to absolute locations. Furthermore, EEG electrode locations that
    do not fit to a sphere will distort the layout when computing the z-values.
    If you are not sure about the channel locations in the header, using a
    montage is encouraged.</p></div>

### EGI simple binary (.egi)

EGI simple binary files can be read using `mne.io.read_raw_egi`.
EGI raw files are simple binary files with a header and can be exported by the
EGI Netstation acquisition software.



### EGI MFF (.mff)

EGI MFF files can be read with `mne.io.read_raw_egi`.



### EEGLAB files (.set, .fdt)

EEGLAB .set files (which sometimes come with a separate .fdt file) can be read
using `mne.io.read_raw_eeglab` and `mne.read_epochs_eeglab`.



### Nicolet (.data)

These files can be read with `mne.io.read_raw_nicolet`.



### eXimia EEG data (.nxe)

EEG data from the Nexstim eXimia system can be read with `mne.io.read_raw_eximia`.



### Persyst EEG data (.lay, .dat)

EEG data from the Persyst system can be read with `mne.io.read_raw_persyst`.

Note that subject metadata may not be properly imported because Persyst
sometimes changes its specification from version to version. Please let us know
if you encounter a problem.

### Nihon Kohden EEG data (.eeg, .21e, .pnt, .log)

EEG data from the Nihon Kohden (NK) system can be read using the `mne.io.read_raw_nihon` function.

Files with the following extensions will be read:

- The `.eeg` file contains the actual raw EEG data.
- The `.pnt` file contains metadata related to the recording such as the measurement date.
- The `.log` file contains annotations for the recording.
- The `.21e` file contains channel and electrode information.

Reading `.11d`, `.cmt`, `.cn2`, and `.edf` files is currently not supported.

Note that not all subject metadata may be properly read because NK changes the specification sometimes from version to version. Please let us know if you encounter a problem.

### XDF data (.xdf, .xdfz)

MNE-Python does not support loading [XDF](https://github.com/sccn/xdf/wiki/Specifications) files out of the box, because the inherent flexibility of the XDF format makes it difficult to provide a one-size-fits-all function. For example, XDF supports signals from various modalities recorded with different sampling rates. However, it is relatively straightforward to import only a specific stream (such as EEG signals) using the [pyxdf](https://github.com/xdf-modules/pyxdf) package.

A more sophisticated version, which supports selection of specific streams as well as converting marker streams into annotations, is available in [MNELAB](https://github.com/cbrnr/mnelab). If you want to use this functionality in a script, MNELAB records its history (View - History), which contains all commands required to load an XDF file after successfully loading that file with the graphical user interface.


### Setting EEG references

The preferred method for applying an EEG reference in MNE is `mne.set_eeg_reference`, or equivalent instance methods like `raw.set_eeg_reference()`. By default, the data are assumed to already be properly referenced.


### Reading electrode locations and head shapes for EEG recordings

Some EEG formats (e.g., EGI, EDF/EDF+, BDF) contain neither electrode locations nor head shape digitization information. Therefore, this information has to be provided separately. For that purpose, all raw instances have a `mne.io.Raw.set_montage` method to set electrode locations.

When using locations of fiducial points, the digitization data are converted to the MEG head coordinate system employed in the MNE software.

## 3. Importing Data from fNIRS Devices

fNIRS devices consist of two kinds of optodes: light sources (AKA "emitters" or
"transmitters") and light detectors (AKA "receivers"). Channels are defined as
source-detector pairs, and channel locations are defined as the midpoint
between source and detector.

MNE-Python provides functions for reading fNIRS data and optode locations from
several file formats. Regardless of the device manufacturer or file format,
MNE-Python's fNIRS functions will internally store the measurement data and its
metadata in the same way (e.g., data values are always converted into SI
units). Supported measurement types include amplitude, optical density,
oxyhaemoglobin concentration, and deoxyhemoglobin concentration (for continuous
wave fNIRS), and additionally AC amplitude and phase (for
frequency domain fNIRS).

<div class="alert alert-danger"><h4>Warning</h4><p>MNE-Python stores metadata internally with a specific structure,
             and internal functions expect specific naming conventions.
             Manual modification of channel names and metadata
             is not recommended.</p></div>

## Standardized Data

### SNIRF (.snirf)

The Shared Near Infrared Spectroscopy Format
([SNIRF](https://github.com/fNIRS/snirf/blob/master/snirf_specification.md)_)
is designed by the fNIRS community in an effort to facilitate
sharing and analysis of fNIRS data. And is the official format of the
Society for functional near-infrared spectroscopy (SfNIRS).
The manufacturers Gowerlabs, NIRx, Kernel, Artinis, and Cortivision
export data in the SNIRF format, and these files can be imported in to MNE.
SNIRF is the preferred format for reading data in to MNE-Python.
Data stored in the SNIRF format can be read in
using `mne.io.read_raw_snirf`.

<div class="alert alert-info"><h4>Note</h4><p>The SNIRF format has provisions for many different types of fNIRS
          recordings. MNE-Python currently only supports reading continuous
          wave or haemoglobin data stored in the .snirf format.</p></div>
          
          
#### Specifying the coordinate system

There are a variety of coordinate systems used to specify the location of
sensors (see `tut-source-alignment` for details). Where possible the
coordinate system will be determined automatically when reading a SNIRF file.
However, sometimes this is not possible and you must manually specify the
coordinate frame the optodes are in. This is done using the ``optode_frame``
argument when loading data.


**Correct the Table** #########################
=======  ==================  =================
Vendor   Model               ``optode_frame``
=======  ==================  =================
NIRx     ICBM-152 MNI        mri
Kernel   ICBM 2009b          mri
=======  ==================  =================
**Correct the Table** #########################

The coordinate system is automatically detected for Gowerlabs SNIRF files.

## Continuous Wave Devices

### NIRx (directory or hdr)

NIRx produce continuous wave fNIRS devices. NIRx recordings can be read in using `mne.io.read_raw_nirx`.
The NIRx device stores data directly to a directory with multiple file types, MNE-Python extracts the appropriate information from each file. MNE-Python only supports NIRx files recorded with NIRStar version 15.0 and above and Aurora version 2021 and above. MNE-Python supports reading data from NIRScout and NIRSport devices.

### Hitachi (.csv)

Hitachi produce continuous wave fNIRS devices. Hitachi fNIRS recordings can be read using `mne.io.read_raw_hitachi`. No optode information is stored so you'll need to set the montage manually, see the Notes section of `mne.io.read_raw_hitachi`.

## Frequency Domain Devices


### BOXY (.txt)

BOXY recordings can be read in using `mne.io.read_raw_boxy`. The BOXY software and ISS Imagent I and II devices are frequency domain systems that store data in a single ``.txt`` file containing what they call (with MNE-Python's name for that type of data in parens):

- **DC**
    All light collected by the detector (``fnirs_cw_amplitude``)
- **AC**
    High-frequency modulated light intensity (``fnirs_fd_ac_amplitude``)
- **Phase**
    Phase of the modulated light (``fnirs_fd_phase``)

DC data is stored as the type ``fnirs_cw_amplitude`` because it collects both the modulated and any unmodulated light, and hence is analogous to what is collected by continuous wave systems such as NIRx. This helps with conformance to SNIRF standard types.

These raw data files can be saved by the acquisition devices as parsed or unparsed ``.txt`` files, which affects how the data in the file is organised. MNE-Python will read either file type and extract the raw DC, AC, and Phase data. If triggers are sent using the ``digaux`` port of the recording hardware, MNE-Python will also read the ``digaux`` data and create annotations for any triggers.

## Custom Data Import

### Loading legacy data in CSV or TSV format

<div class="alert alert-danger"><h4>Warning</h4><p>This method is not supported and users are discouraged to use it.
             You should convert your data to the
             [SNIRF](https://github.com/fNIRS/snirf) format using the tools
             provided by the Society for functional Near-Infrared Spectroscopy,
             and then load it using :func:`mne.io.read_raw_snirf`.</p></div>

fNIRS measurements may be stored in a non-standardised format that is not supported by MNE-Python and cannot be converted easily into SNIRF. This legacy data is often in CSV or TSV format, we show here a way to load it even though it is not officially supported by MNE-Python due to the lack of standardisation of the file format (the naming and ordering of channels, the type and scaling of data, and specification of sensor positions varies between each vendor). You will likely have to adapt this depending on the system from which your CSV originated.

In [ ]:
import numpy as np
import pandas as pd

import mne

First, we generate an example CSV file which will then be loaded in to MNE-Python. This step would be skipped if you have actual data you wish to load. We simulate 16 channels with 100 samples of data and save this to a file called fnirs.csv.

In [ ]:
pd.DataFrame(np.random.normal(size=(16, 100))).to_csv("fnirs.csv")

Next, we will load the example CSV file.

In [ ]:
data = pd.read_csv("fnirs.csv")

Then, the metadata must be specified manually as the CSV file does not
contain information about channel names, types, sample rate etc.

<div class="alert alert-danger"><h4>Warning</h4><p>In MNE-Python the naming of channels MUST follow the structure
             ``S#_D# type`` where # is replaced by the appropriate source and
             detector numbers and type is either ``hbo``, ``hbr`` or the
             wavelength.</p></div>

In [ ]:
ch_names = [
    "S1_D1 hbo",
    "S1_D1 hbr",
    "S2_D1 hbo",
    "S2_D1 hbr",
    "S3_D1 hbo",
    "S3_D1 hbr",
    "S4_D1 hbo",
    "S4_D1 hbr",
    "S5_D2 hbo",
    "S5_D2 hbr",
    "S6_D2 hbo",
    "S6_D2 hbr",
    "S7_D2 hbo",
    "S7_D2 hbr",
    "S8_D2 hbo",
    "S8_D2 hbr",
]
ch_types = [
    "hbo",
    "hbr",
    "hbo",
    "hbr",
    "hbo",
    "hbr",
    "hbo",
    "hbr",
    "hbo",
    "hbr",
    "hbo",
    "hbr",
    "hbo",
    "hbr",
    "hbo",
    "hbr",
]
sfreq = 10.0  # in Hz

Finally, the data can be converted in to an MNE-Python data structure. The metadata above is used to create an `mne.Info` data structure, and this is combined with the data to create an MNE-Python
:class:`mne.io.Raw` object.

In [ ]:
info = mne.create_info(ch_names=ch_names, ch_types=ch_types, sfreq=sfreq)
raw = mne.io.RawArray(data, info, verbose=True)

### Applying standard sensor locations to imported data

Having information about optode locations may assist in your analysis. Beyond the general benefits this provides (e.g. creating regions of interest, etc), this is may be particularly important for fNIRS as information about the optode locations is required to convert the optical density data in to an estimate of the haemoglobin concentrations. MNE-Python provides methods to load standard sensor configurations (montages) from some vendors, and this is demonstrated below.

Below is an example of how to load the optode positions for an Artinis
OctaMon device.

<div class="alert alert-info"><h4>Note</h4><p>It is also possible to create a custom montage from a file for
          fNIRS with `mne.channels.read_custom_montage` by setting
          `coord_frame` to `'mri'`.</p></div>

In [ ]:
montage = mne.channels.make_standard_montage("artinis-octamon")
raw.set_montage(montage)

# View the position of optodes in 2D to confirm the positions are correct.
raw.plot_sensors()

To validate the positions were loaded correctly it is also possible to view the location of the sources (red), detectors (black), and channels (white lines and orange dots) in a 3D representation. The ficiduals are marked in blue, green and red. See `Source alignment and coordinate frames` for more details.

In [ ]:
subjects_dir = mne.datasets.sample.data_path() / "subjects"
mne.datasets.fetch_fsaverage(subjects_dir=subjects_dir)

brain = mne.viz.Brain(
    "fsaverage", subjects_dir=subjects_dir, alpha=0.5, cortex="low_contrast"
)
brain.add_head()
brain.add_sensors(raw.info, trans="fsaverage")
brain.show_view(azimuth=90, elevation=90, distance=500)

## 4. Working woth CTF Data: the Brainstorm Auditory Dataset

Here we compute the evoked from raw for the auditory Brainstorm
tutorial dataset. For comparison, see the associated [brainstorm site](https://neuroimage.usc.edu/brainstorm/Tutorials/Auditory).

Experiment:

    - One subject, 2 acquisition runs 6 minutes each.
    - Each run contains 200 regular beeps and 40 easy deviant beeps.
    - Random ISI: between 0.7s and 1.7s seconds, uniformly distributed.
    - Button pressed when detecting a deviant with the right index finger.

The specifications of this dataset were discussed initially on the [FieldTrip bug tracker](http://bugzilla.fieldtriptoolbox.org/show_bug.cgi?id=2300).

In [ ]:
import numpy as np
import pandas as pd

import mne
from mne import combine_evoked
from mne.datasets.brainstorm import bst_auditory
from mne.io import read_raw_ctf
from mne.minimum_norm import apply_inverse

To reduce memory consumption and running time, some of the steps are precomputed. To run everything from scratch change ``use_precomputed`` to ``False``. With ``use_precomputed = False`` running time of this script can be several minutes even on a fast computer.

In [ ]:
use_precomputed = True

The data was collected with a CTF 275 system at 2400 Hz and low-pass filtered at 600 Hz. Here the data and empty room data files are read to construct instances of `mne.io.Raw`.

In [ ]:
data_path = bst_auditory.data_path()

subject = "bst_auditory"
subjects_dir = data_path / "subjects"

raw_fname1 = data_path / "MEG" / subject / "S01_AEF_20131218_01.ds"
raw_fname2 = data_path / "MEG" / subject / "S01_AEF_20131218_02.ds"
erm_fname = data_path / "MEG" / subject / "S01_Noise_20131218_01.ds"

In the memory saving mode we use ``preload=False`` and use the memory efficient IO which loads the data on demand. However, filtering and some other functions require the data to be preloaded into memory.

In [ ]:
raw = read_raw_ctf(raw_fname1)
n_times_run1 = raw.n_times

# Here we ignore that these have different device<->head transforms
mne.io.concatenate_raws([raw, read_raw_ctf(raw_fname2)], on_mismatch="ignore")
raw_erm = read_raw_ctf(erm_fname)

The data array consists of 274 MEG axial gradiometers, 26 MEG reference
sensors and 2 EEG electrodes (Cz and Pz). In addition:

  - 1 stim channel for marking presentation times for the stimuli
  - 1 audio channel for the sent signal
  - 1 response channel for recording the button presses
  - 1 ECG bipolar
  - 2 EOG bipolar (vertical and horizontal)
  - 12 head tracking channels
  - 20 unused channels

Notice also that the digitized electrode positions (stored in a .pos file)
were automatically loaded and added to the `mne.io.Raw` object.

The head tracking channels and the unused channels are marked as misc
channels. Here we define the EOG and ECG channels.

In [ ]:
raw.set_channel_types({"HEOG": "eog", "VEOG": "eog", "ECG": "ecg"})
if not use_precomputed:
    # Leave out the two EEG channels for easier computation of forward.
    raw.pick(["meg", "stim", "misc", "eog", "ecg"]).load_data()

For noise reduction, a set of bad segments have been identified and stored in csv files. The bad segments are later used to reject epochs that overlap with them. The file for the second run also contains some saccades. The saccades are removed by using SSP. We use pandas to read the data from the csv files. You can also view the files with your favorite text editor.

In [ ]:
annotations_df = pd.DataFrame()
offset = n_times_run1
for idx in [1, 2]:
    csv_fname = data_path / "MEG" / "bst_auditory" / f"events_bad_0{idx}.csv"
    df = pd.read_csv(csv_fname, header=None, names=["onset", "duration", "id", "label"])
    print(f"Events from run {idx}:")
    print(df)

    df["onset"] += offset * (idx - 1)
    annotations_df = pd.concat([annotations_df, df], axis=0)

saccades_events = df[df["label"] == "saccade"].values[:, :3].astype(int)

# Conversion from samples to times:
onsets = annotations_df["onset"].values / raw.info["sfreq"]
durations = annotations_df["duration"].values / raw.info["sfreq"]
descriptions = annotations_df["label"].values

annotations = mne.Annotations(onsets, durations, descriptions)
raw.set_annotations(annotations)
del onsets, durations, descriptions

Here we compute the saccade and EOG projectors for magnetometers and add them to the raw data. The projectors are added to both runs.

In [ ]:
saccade_epochs = mne.Epochs(
    raw,
    saccades_events,
    1,
    0.0,
    0.5,
    preload=True,
    baseline=(None, None),
    reject_by_annotation=False,
)

projs_saccade = mne.compute_proj_epochs(
    saccade_epochs, n_mag=1, n_eeg=0, desc_prefix="saccade"
)
if use_precomputed:
    proj_fname = data_path / "MEG" / "bst_auditory" / "bst_auditory-eog-proj.fif"
    projs_eog = mne.read_proj(proj_fname)[0]
else:
    projs_eog, _ = mne.preprocessing.compute_proj_eog(raw.load_data(), n_mag=1, n_eeg=0)
raw.add_proj(projs_saccade)
raw.add_proj(projs_eog)
del saccade_epochs, saccades_events, projs_eog, projs_saccade  # To save memory

Visually inspect the effects of projections. Click on 'proj' button at the bottom right corner to toggle the projectors on/off. EOG events can be plotted by adding the event list as a keyword argument. As the bad segments and saccades were added as annotations to the raw data, they are plotted as well.

In [ ]:
raw.plot()

Typical preprocessing step is the removal of power line artifact (50 Hz or 60 Hz). Here we notch filter the data at 60, 120 and 180 to remove the original 60 Hz artifact and the harmonics. The power spectra are plotted
before and after the filtering to show the effect. The drop after 600 Hz appears because the data was filtered during the acquisition. In memory saving mode we do the filtering at evoked stage, which is not something you usually would do.

In [ ]:
if not use_precomputed:
    raw.compute_psd(tmax=np.inf, picks="meg").plot(
        picks="data", exclude="bads", amplitude=False
    )
    notches = np.arange(60, 181, 60)
    raw.notch_filter(notches, phase="zero-double", fir_design="firwin2")
    raw.compute_psd(tmax=np.inf, picks="meg").plot(
        picks="data", exclude="bads", amplitude=False
    )

We also lowpass filter the data at 100 Hz to remove the hf components.

In [ ]:
if not use_precomputed:
    raw.filter(
        None,
        100.0,
        h_trans_bandwidth=0.5,
        filter_length="10s",
        phase="zero-double",
        fir_design="firwin2",
    )

Epoching and averaging. First some parameters are defined and events extracted from the stimulus channel (UPPT001). The rejection thresholds are defined as peak-to-peak values and are in T / m for gradiometers, T for magnetometers and V for EOG and EEG channels.

In [ ]:
tmin, tmax = -0.1, 0.5
event_id = dict(standard=1, deviant=2)
reject = dict(mag=4e-12, eog=250e-6)
# find events
events = mne.find_events(raw, stim_channel="UPPT001")

The event timing is adjusted by comparing the trigger times on detected sound onsets on channel UADC001-4408.

In [ ]:
sound_data = raw[raw.ch_names.index("UADC001-4408")][0][0]
onsets = np.where(np.abs(sound_data) > 2.0 * np.std(sound_data))[0]
min_diff = int(0.5 * raw.info["sfreq"])
diffs = np.concatenate([[min_diff + 1], np.diff(onsets)])
onsets = onsets[diffs > min_diff]
assert len(onsets) == len(events)
diffs = 1000.0 * (events[:, 0] - onsets) / raw.info["sfreq"]
print(f"Trigger delay removed (μ ± σ): {np.mean(diffs):0.1f} ± {np.std(diffs):0.1f} ms")
events[:, 0] = onsets
del sound_data, diffs

We mark a set of bad channels that seem noisier than others. This can also be done interactively with ``raw.plot`` by clicking the channel name (or the line). The marked channels are added as bad when the browser window is closed.

In [ ]:
raw.info["bads"] = ["MLO52-4408", "MRT51-4408", "MLO42-4408", "MLO43-4408"]

The epochs (trials) are created for MEG channels. First we find the picks for MEG and EOG channels. Then the epochs are constructed using these picks. The epochs overlapping with annotated bad segments are also rejected by default. To turn off rejection by bad segments (as was done earlier with saccades) you can use keyword ``reject_by_annotation=False``.

In [ ]:
epochs = mne.Epochs(
    raw,
    events,
    event_id,
    tmin,
    tmax,
    picks=["meg", "eog"],
    baseline=(None, 0),
    reject=reject,
    preload=False,
    proj=True,
)

We only use first 40 good epochs from each run. Since we first drop the bad epochs, the indices of the epochs are no longer same as in the original epochs collection. Investigation of the event timings reveals that first epoch from the second run corresponds to index 182.

In [ ]:
epochs.drop_bad()

# avoid warning about concatenating with annotations
epochs.set_annotations(None)

epochs_standard = mne.concatenate_epochs(
    [epochs["standard"][range(40)], epochs["standard"][182:222]]
)
epochs_standard.load_data()  # Resampling to save memory.
epochs_standard.resample(600, npad="auto")
epochs_deviant = epochs["deviant"].load_data()
epochs_deviant.resample(600, npad="auto")
del epochs

The averages for each conditions are computed.

In [ ]:
evoked_std = epochs_standard.average()
evoked_dev = epochs_deviant.average()
del epochs_standard, epochs_deviant

Typical preprocessing step is the removal of power line artifact (50 Hz or 60 Hz). Here we lowpass filter the data at 40 Hz, which will remove all line artifacts (and high frequency information). Normally this would be done to raw data (with `mne.io.Raw.filter`), but to reduce memory consumption of this tutorial, we do it at evoked stage. (At the raw stage, you could alternatively notch filter with `mne.io.Raw.notch_filter`.)

In [ ]:
for evoked in (evoked_std, evoked_dev):
    evoked.filter(l_freq=None, h_freq=40.0, fir_design="firwin")

Here we plot the ERF of standard and deviant conditions. In both conditions we can see the P50 and N100 responses. The mismatch negativity is visible only in the deviant condition around 100-200 ms. P200 is also visible around 170 ms in both conditions but much stronger in the standard condition. P300 is visible in deviant condition only (decision making in preparation of the button press). You can view the topographies from a certain time span by painting an area with clicking and holding the left mouse button.

In [ ]:
evoked_std.plot(window_title="Standard", gfp=True, time_unit="s")
evoked_dev.plot(window_title="Deviant", gfp=True, time_unit="s")

Show activations as topography figures.

In [ ]:
times = np.arange(0.05, 0.301, 0.025)
fig = evoked_std.plot_topomap(times=times)
fig.suptitle("Standard")

In [ ]:
fig = evoked_dev.plot_topomap(times=times)
fig.suptitle("Deviant")

We can see the MMN effect more clearly by looking at the difference between the two conditions. P50 and N100 are no longer visible, but MMN/P200 and P300 are emphasised.

In [ ]:
evoked_difference = combine_evoked([evoked_dev, evoked_std], weights=[1, -1])
evoked_difference.plot(window_title="Difference", gfp=True, time_unit="s")

Source estimation. We compute the noise covariance matrix from the empty room measurement and use it for the other runs.

In [ ]:
reject = dict(mag=4e-12)
cov = mne.compute_raw_covariance(raw_erm, reject=reject)
cov.plot(raw_erm.info)
del raw_erm

The transformation is read from a file:

In [ ]:
trans_fname = data_path / "MEG" / "bst_auditory" / "bst_auditory-trans.fif"
trans = mne.read_trans(trans_fname)

To save time and memory, the forward solution is read from a file. Set ``use_precomputed=False`` in the beginning of this script to build the forward solution from scratch. The head surfaces for constructing a BEM solution are read from a file. Since the data only contains MEG channels, we only need the inner skull surface for making the forward solution.

In [ ]:
if use_precomputed:
    fwd_fname = data_path / "MEG" / "bst_auditory" / "bst_auditory-meg-oct-6-fwd.fif"
    fwd = mne.read_forward_solution(fwd_fname)
else:
    src = mne.setup_source_space(
        subject, spacing="ico4", subjects_dir=subjects_dir, overwrite=True
    )
    model = mne.make_bem_model(
        subject=subject, ico=4, conductivity=[0.3], subjects_dir=subjects_dir
    )
    bem = mne.make_bem_solution(model)
    fwd = mne.make_forward_solution(evoked_std.info, trans=trans, src=src, bem=bem)

inv = mne.minimum_norm.make_inverse_operator(evoked_std.info, fwd, cov)
snr = 3.0
lambda2 = 1.0 / snr**2
del fwd

The sources are computed using dSPM method and plotted on an inflated brain surface. For interactive controls over the image, use keyword ``time_viewer=True``. Standard condition.

In [ ]:
stc_standard = mne.minimum_norm.apply_inverse(evoked_std, inv, lambda2, "dSPM")
brain = stc_standard.plot(
    subjects_dir=subjects_dir,
    subject=subject,
    surface="inflated",
    time_viewer=False,
    hemi="lh",
    initial_time=0.1,
    time_unit="s",
)
del stc_standard, brain

Deviant condition.

In [ ]:
stc_deviant = mne.minimum_norm.apply_inverse(evoked_dev, inv, lambda2, "dSPM")
brain = stc_deviant.plot(
    subjects_dir=subjects_dir,
    subject=subject,
    surface="inflated",
    time_viewer=False,
    hemi="lh",
    initial_time=0.1,
    time_unit="s",
)
del stc_deviant, brain

Difference.

In [ ]:
stc_difference = apply_inverse(evoked_difference, inv, lambda2, "dSPM")
brain = stc_difference.plot(
    subjects_dir=subjects_dir,
    subject=subject,
    surface="inflated",
    time_viewer=False,
    hemi="lh",
    initial_time=0.15,
    time_unit="s",
)

## 5. Importing Data from Eyetracking Devices

Eyetracking devices record a persons point of gaze, usually in relation to a
screen. Typically, gaze position (also referred to as eye or pupil position)
and pupil size are recorded as separate channels. This section describes how to
read data from supported eyetracking manufacturers.

MNE-Python provides functions for reading eyetracking data. When possible,
MNE-Python will internally convert and store eyetracking data according to an
SI unit (for example radians for position data, and meters for pupil size).

<div class="alert alert-info"><h4>Note</h4><p>If you have eye tracking data in a format that MNE does not support
          yet, you can try reading it using other tools and create an MNE
          object from a numpy array. Then you can use
          :func:`mne.preprocessing.eyetracking.set_channel_types_eyetrack`
          to assign the correct eyetrack channel types.</p></div>

Some MNE functions may not be available to eyetracking and other
             physiological data, because MNE does not consider them to be data
             channels. See the `glossary` for more information.

### SR Research (Eyelink) (.asc)

<div class="alert alert-info"><h4>Note</h4><p>MNE-Python currently only supports reading Eyelink eyetracking data
          stored in the ASCII (.asc) format.</p></div>

Eyelink recordings are stored in the Eyelink Data Format (EDF; .edf), which are binary files and thus relatively complex to support. To make the data in EDF files accessible, Eyelink provides the application EDF2ASC, which converts EDF files to a plain text ASCII format (.asc). These files can be imported into MNE using `mne.io.read_raw_eyelink`.

<div class="alert alert-info"><h4>Note</h4><p>The Eyelink Data Format (EDF), should not be confused
          with the European Data Format, the common EEG data format that also
          uses the .edf extension.</p></div>

Supported measurement types from Eyelink files include eye position, pupil size, saccadic velocity, resolution, and head position (for recordings collected in remote mode). Eyelink files often report ocular events (blinks, saccades, and fixations), MNE will store these events as `mne.Annotations`. Blink annotation descriptions will be ``'BAD_blink'``. For more information on the various measurement types that can be present in Eyelink files. read below.

### Eye Position Data

Eyelink samples can report eye position data in pixels, units of visual degrees, or as raw pupil coordinates. Samples are written as (x, y) coordinate pairs (or two pairs for binocular data). The type of position data present in an ASCII file will be detected automatically by MNE. The three types of position data are explained below.

### Gaze

Gaze position data report the estimated (x, y) pixel coordinates of the participants's gaze on the stimulus screen, compensating for head position changes and distance from  the screen. This datatype may be preferable if you are interested in knowing where the participant was looking at on the stimulus screen. The default (0, 0) location for Eyelink systems is at the top left of the screen.

This may be best demonstrated with an example. In the file plotted below, eyetracking data was recorded while the participant read text on a display. In this file, as the participant read the each line from left to right, the x-coordinate increased. When the participant moved their gaze down to read a new line, the y-coordinate *increased*, which is why the ``ypos_right`` channel in the plot below increases over time (for example, at about 4-seconds, and at about 8-seconds).

In [ ]:
from mne.datasets import misc
from mne.io import read_raw_eyelink

In [ ]:
fpath = misc.data_path() / "eyetracking" / "eyelink"
raw = read_raw_eyelink(fpath / "px_textpage_ws.asc", create_annotations=["blinks"])
custom_scalings = dict(eyegaze=1e3)
raw.pick(picks="eyetrack").plot(scalings=custom_scalings)

**important**: 

```
The (0, 0) pixel coordinates are at the top-left of the trackable area of the screen. Gaze towards lower areas of the screen will yield a relatively higher y-coordinate.
```

Note that we passed a custom `dict` to the ``'scalings'`` argument of
`mne.io.Raw.plot`. This is because MNE's default plot scalings for eye
position data are calibrated for HREF data, which are stored in radians
(read below).

### Head-Referenced Eye Angle (HREF)

HREF position data measures eye rotation angles relative to the head. It does not take into account changes in subject head position and angle, or distance from the stimulus screen. This datatype might be preferable for analyses that are interested in eye movement velocities and amplitudes, or for simultaneous and EEG/MEG eyetracking recordings where eye position data are used to identify EOG artifacts.

HREF coordinates are stored in the ASCII file as integer values, with 260 or more units per visual degree, however MNE will convert and store these coordinates in radians. The (0, 0) point of HREF data is arbitrary, as the relationship between the screen position and the coordinates changes as the subject's head moves.

Below is the same text reading recording that we plotted above, except a new ASCII file was generated, this time using HREF eye position data.

In [ ]:
fpath = misc.data_path() / "eyetracking" / "eyelink"
raw = read_raw_eyelink(fpath / "HREF_textpage_ws.asc", create_annotations=["blinks"])
raw.pick(picks="eyetrack").plot()

#### Pupil Position

Pupil position data contains (x, y) coordinate pairs from the eye camera. It has not been converted to pixels (gaze) or eye angles (HREF). Most use cases do not require this data type, and caution should be taken when analyzing raw pupil positions. Note that when plotting data from a ``Raw`` object containing raw pupil position data, the plot scalings will likely be incorrect. You can pass custom scalings into the ``scalings`` parameter of `mne.io.Raw.plot` so that the signals are legible when plotting.

<div class="alert alert-danger"><h4>Warning</h4><p>If a calibration was not performed prior to data collection, the
             EyeLink system cannot convert raw pupil position data to pixels
             (gaze) or eye angle (HREF).</p></div>

### Pupil Size Data

Pupil size is measured by the EyeLink system at up to 500 samples per second. It may be reported as pupil *area*, or pupil *diameter* (i.e. the diameter of a circle/ellipse model fit to the pupil area). Which of these datatypes you get is specified by your recording- and/or your EDF2ASC settings. The pupil size data is not calibrated and reported in arbitrary units. Typical pupil *area* data range between 800 to 2000 units, with a precision of 1 unit, while pupil *diameter* data range between 1800-3000 units.

### Velocity, resolution, and head position data

Eyelink files can produce data on saccadic velocity, resolution, and head position for each sample in the file. MNE will read in these data if they are present in the file, but will label their channel types as ``'misc'``.

**Warning**:
```
Eyelink's EDF2ASC API allows for modification of the data and format that is converted to ASCII. However, MNE-Python assumes a specific structure, which the default parameters of EDF2ASC follow. ASCII files should be tab-deliminted, and both Samples and Events should be output. If the data were recorded at 2000Hz, timestamps should be floating point numbers. Manual modification of ASCII conversion via EDF2ASC is not recommended.       
```